In [ ]:
# Figure 1: panels a and b

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import moment
import variogram_helper_functions
from scipy.ndimage import gaussian_filter
import matplotlib


matplotlib.rcParams["font.family"] = "Roboto"
matplotlib.rcParams["font.sans-serif"] = ["Roboto"]  # For non-unicode text
matplotlib.rcParams['axes.labelsize'] = 18
matplotlib.rcParams['axes.titlesize'] = 18
matplotlib.rcParams['xtick.labelsize'] = 18
matplotlib.rcParams['ytick.labelsize'] = 18
matplotlib.rcParams['legend.fontsize'] = 18
matplotlib.rcParams['legend.facecolor'] = 'w'


def generate_rough_and_smooth_surfaces(size):
     # Generate a rough field using random noise
    random_field = np.random.randn(size,size)#*100

    # Apply a Gaussian filter to create a smooth field
    sigma = 10  # Standard deviation for the Gaussian kernel (controls smoothness)
    rough_field = gaussian_filter(random_field, sigma=0.6)
    #rough_field = random_field
    smooth_field = gaussian_filter(random_field, sigma=sigma)
    smooth_field = (smooth_field - np.mean(smooth_field)) / np.std(smooth_field) * np.std(rough_field) + np.mean(rough_field)
    print('min max of rough surface = ',np.min(rough_field),np.max(rough_field))
    print('min max of smooth surface = ',np.min(smooth_field),np.max(smooth_field))
    return rough_field, smooth_field


# Compute statistical moments
def compute_moments(field, name):
    mean = np.mean(field)
    variance = np.var(field)
    skewness = moment(field.flatten(), moment=3) / np.std(field)**3  # Third moment (skewness)
    kurtosis = moment(field.flatten(), moment=4) / np.var(field)**2  # Fourth moment (kurtosis)
    fifth_moment = moment(field.flatten(), moment=5) / np.std(field)**5  # Fifth moment

    print(f"{name} Field Moments:")
    print(f"  Mean: {mean:.4f}")
    print(f"  Variance: {variance:.4f}")
    print(f"  Skewness: {skewness:.4f}")
    print(f"  Kurtosis: {kurtosis:.4f}")
    print(f"  Fifth Moment: {fifth_moment:.4f}")
    print("-" * 40)

######----------------------------------------------------------------------
np.random.seed(25)
# Set grid size
size = 200
rough_field, smooth_field = generate_rough_and_smooth_surfaces(size)

compute_moments(rough_field, "Rough")
compute_moments(smooth_field, "Smooth")


# Plot the two fields
fig, axes = plt.subplots(1, 2, figsize=(10,5))

#Rough field plot
im1 = axes[0].contourf(rough_field, cmap='viridis', origin='lower', levels=np.arange(-2., 2.05, 0.05), extend='both')
axes[0].set_title("Field A (rough)")
axes[0].set_xlabel("distance (km)")
axes[0].set_ylabel("distance (km)")
axes[0].set_aspect('equal')

# Add colorbar for rough field
cbar1 = fig.colorbar(im1, ax=axes[0], orientation='horizontal', fraction=0.046, pad=0.2)
#cbar1.set_label(label='',size=5)
cbar1.ax.tick_params(labelsize=11)  

# Smooth field plot
im2 = axes[1].contourf(smooth_field, cmap='viridis', origin='lower', levels=np.arange(-2., 2.05, 0.05), extend='both')
axes[1].set_title("Field B (smooth)")
axes[1].set_xlabel("distance (km)")
axes[1].set_ylabel("distance (km)")
axes[1].set_aspect('equal')

# Add colorbar for smooth field
cbar2 = fig.colorbar(im2, ax=axes[1], orientation='horizontal', fraction=0.046, pad=0.2)
#cbar2.set_label(label='',size=5)
cbar2.ax.tick_params(labelsize=11)  

plt.tight_layout()
plt.savefig('paper_1_new_fig1_panels_a_b.png',dpi=150)

In [ ]:
# Figure 1: panels c and d

import variogram_helper_functions


def plot_variogram_toy(TEMP_DATA,SAMPLE_SIZE,BIN_FUNCTION,ESTIMATOR):
    y_dim, x_dim     = np.shape(TEMP_DATA)
    coords = variogram_helper_functions.produce_random_coords(x_dim,y_dim,SAMPLE_SIZE)                           
    nonnan_coords, nonnan_values = variogram_helper_functions.get_values_at_random_coords(TEMP_DATA, coords)
    max_lag = np.sqrt(x_dim**2 + y_dim**2)/2.0
    V, x1, y1 = variogram_helper_functions.make_variogram(nonnan_coords, nonnan_values, 50, max_lag, DX = 1.0, BIN_FUNCTION=BIN_FUNCTION,ESTIMATOR=ESTIMATOR)
    return V, x1, y1

nsamples = 10000

V1, bins1, vario1  = plot_variogram_toy(rough_field,nsamples,'even','matheron')
print('----')
V2, bins2, vario2  = plot_variogram_toy(smooth_field,nsamples,'even','matheron')
print('------------\n\n')


S1, bins_str_func1, str_func1  = plot_variogram_toy(rough_field,nsamples,'even','first_order_structure_function')
print('----')
S2, bins_str_func2, str_func2  = plot_variogram_toy(smooth_field,nsamples,'even','first_order_structure_function')
print('------------\n\n')


fig, axes    = plt.subplots(1,2,figsize=(10,5))

axes[0].plot(bins_str_func1, str_func1, label='First-Order SF',color='g',linewidth=3)
axes[0].plot(bins1, vario1, label='Variogram',color='#999999',linewidth=3)
#axes[0].set_title('Variogram/Order 1 Structure Function')
axes[0].set_xlabel('Spatial lag (km)')
axes[0].set_ylabel('Variogram/First-Order Structure Function')# $(arbitrary-unit^{2})$')
axes[0].set_ylim([0,1.3])

axes[1].plot(bins_str_func2, str_func2, label='First-Order SF',color='g',linewidth=3)
axes[1].plot(bins2, vario2, label='Variogram',color='#999999',linewidth=3)
#axes[1].set_title('Variogram')
axes[1].set_xlabel('Spatial lag (km)')
axes[1].set_ylabel('Variogram/First-Order Structure Function')# $(arbitrary-unit^{2})$')
axes[1].set_ylim([0,1.3])

axes[0].legend(fontsize=15)
axes[1].legend(fontsize=15)

axes[0].grid(True, which="both", ls="--")  # Grid for both axes in log scale
axes[1].grid(True, which="both", ls="--")  # Grid for both axes in log scale


plt.tight_layout()
plt.savefig('paper_1_new_fig1_panels_c_d.png',dpi=150,bbox_inches='tight')

In [ ]:
# Figure 1: panels e and f

from scipy.optimize import curve_fit
import matplotlib.ticker as mticker

# Define a linear model for curve fitting
def linear_model(x, a, b):
    return a * x + b

def fit_data(spatial_lags,structure_function_vals):

    log_bins = np.log10(spatial_lags)
    log_exp_variogram = np.log10(structure_function_vals)

    # Fit the data
    params, cov = curve_fit(linear_model, log_bins, log_exp_variogram)
    slope, intercept = params

    slope_var = cov[0, 0]  # Variance of the slope

    # Calculate the fitted line
    fitted_line = linear_model(log_bins, slope, intercept)

    # calculate R^2 
    residuals = log_exp_variogram - linear_model(log_bins,  slope, intercept)
    ss_res    = np.sum(residuals**2)
    #You can get the total sum of squares (ss_tot) with
    ss_tot    = np.sum((log_exp_variogram-np.mean(log_exp_variogram))**2)
    #And finally, the r_squared-value with,
    r_squared = 1 - (ss_res / ss_tot)

    return slope, slope_var, r_squared, fitted_line


def get_fitted_line_strfunc(bins_str_func,str_func,max_bin,q):

    select_bins_str_func = bins_str_func[bins_str_func<=max_bin] 
    select_str_func      = str_func[bins_str_func<=max_bin]

    slope, slope_var, r_squared, fitted_line = fit_data(select_bins_str_func, select_str_func)
    
    label = f"Slope = $\\zeta_{{{q}}}$={slope:.3f}"# ± {slope_var:.2e}, $R^{2}$={np.round(r_squared, 2)}'

    return select_bins_str_func, fitted_line, label


select_bins_str_func1, fitted_line_str_func1, label_str_func1 = get_fitted_line_strfunc(bins_str_func1,str_func1,140,'1')
select_bins_vario1, fitted_line_vario1, label_vario1 = get_fitted_line_strfunc(bins1,vario1,140,'2')


select_bins_str_func2, fitted_line_str_func2, label_str_func2 = get_fitted_line_strfunc(bins_str_func2,str_func2,30,'1')
select_bins_vario2, fitted_line_vario2, label_vario2 = get_fitted_line_strfunc(bins2,vario2,30,'2')


fig, axes = plt.subplots(1,2,figsize=(10,5))
# Plot in loglog scale
data_points1, = axes[0].loglog(bins_str_func1,str_func1, 'o', markersize=3, color='green',label='First-order SF')  # Original data (not log-transformed)
fitted_line1, = axes[0].loglog(select_bins_str_func1, 10**fitted_line_str_func1, color='green', label='First-order SF Linear Fit; '+label_str_func1,linewidth=2.5)#f'Fitted line (y={slope1:.2f}x + {intercept:.2f})')  # Fitted line

data_points1, = axes[0].loglog(bins1,vario1, 'o', markersize=3,color='#999999',label='Variogram')  # Original data (not log-transformed)
fitted_line1, = axes[0].loglog(select_bins_vario1, 10**fitted_line_vario1, color='#999999', label='Variogram Linear Fit; '+label_vario1,linewidth=2.5)#f'Fitted line (y={slope1:.2f}x + {intercept:.2f})')  # Fitted line

axes[0].legend(fontsize = 12)
axes[0].set_xlabel('Spatial lag (km)')
axes[0].set_ylabel('Variogram/First-Order Structure Function')# $(arbitrary-unit^{2})$')
axes[0].set_ylim([0,1.5])
axes[0].grid(True, which="both", ls="--")  # Grid for both axes in log scale
axes[0].yaxis.set_minor_formatter(mticker.ScalarFormatter())
#########

data_points1, = axes[1].loglog(bins_str_func2,str_func2, 'o',  markersize=3, color='green',label='First-order SF')  # Original data (not log-transformed)
fitted_line1, = axes[1].loglog(select_bins_str_func2, 10**fitted_line_str_func2, color='green',label='First-order SF '+label_str_func2,linewidth=2.5)#f'Fitted line (y={slope1:.2f}x + {intercept:.2f})')  # Fitted line

data_points1, = axes[1].loglog(bins2,vario2, 'o', markersize=3, color='#999999',label='Variogram')  # Original data (not log-transformed)
fitted_line1, = axes[1].loglog(select_bins_vario2, 10**fitted_line_vario2, color='#999999', label='Variogram '+label_vario2,linewidth=2.5)#f'Fitted line (y={slope1:.2f}x + {intercept:.2f})')  # Fitted line

axes[1].legend(fontsize = 12)
axes[1].set_xlabel('Spatial lag (km)')
axes[1].set_ylabel('Variogram/First-Order Structure Function')# $(arbitrary-unit^{2})$')
axes[1].set_ylim([0,1.5])
axes[1].grid(True, which="both", ls="--")  # Grid for both axes in log scale


plt.tight_layout()
plt.savefig('paper_1_new_fig1_panels_e_f.png',dpi=150,bbox_inches='tight')

# Save variograms

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import glob
import os
from multiprocessing import Pool, cpu_count
import time
import h5py
import hdf5plugin
import pandas as pd
import xarray as xr
from netCDF4 import Dataset
import cartopy.crs as crs
import random
import skgstat as skg
import matplotlib.ticker as ticker
import read_vars_WRF_RAMS
import variogram_helper_functions
import matplotlib
from skimage import data, filters, measure, morphology


matplotlib.rcParams["font.family"] = "Roboto"
matplotlib.rcParams["font.sans-serif"] = ["Roboto"]  # For non-unicode text
matplotlib.rcParams['axes.labelsize'] = 18
matplotlib.rcParams['axes.titlesize'] = 18
matplotlib.rcParams['xtick.labelsize'] = 18
matplotlib.rcParams['ytick.labelsize'] = 18
matplotlib.rcParams['legend.fontsize'] = 18
 
def save_variogram(VERSION, WHICH_TIME,MODEL_FILE,VARIABLE, SIMULATION, ENSEMBLE_NUMBER, CONDITION_INFO, SAMPLE_SIZE, BIN_WIDTH_IN_KM, DOMAIN, SAVE_CSV, ESTIMATOR='matheron',SAMPLING_STRATEGY='randomly_distributed'):
    """
    VERSION: an additional string to be added to the final csv file; in this cases it is the Version# of RAMS/WRF simulations
    WHICH_TIME: 'start','middle','end' string; grab the files corresponding to these strings
    MODEL_FILE: path to a file; use instead of the 'WHICH_TIME' option 
    VARIABLE:    variable for which we want the variogram; it is a list containing the properties of the storm e.g., ['QV', 0, 'model', '$q_{v}$', '$kg^{2}kg^{-2}$'] 
    SIMULATION: name of the simulation
    CONDITION_INFO:
    SAMPLE_SIZE: # of points to be drawn from the domain 
    DOMAIN: '1' or '2' or '3'
    SAVE_CSV: True or False; whether to save this data or not
    ESTIMATOR='matheron': 'matheron' or 'first_order_structure_function' or 'second_order_structure_function' and so on, upto fifth order
    """
    make_QTC_plot = True
    make_variogram_for_each_storm=False
    
    print('working on domain' ,domain)
    print('working on ',VARIABLE)
    print('working on simulation: ',SIMULATION)
    
    # provide grid spacings in km to multiply with the variogram bin distances
    if DOMAIN=='1':
        dx = 1.6 # km
    if DOMAIN=='2':
        dx=0.4   # km
    if DOMAIN=='3':
        dx=0.1   # km
        
    # which model are we working on; need this for the read_RAMS_WRF_data_file
    if SIMULATION[7]=='W':
        model_name = 'WRF'
        microphysics_scheme = SIMULATION[8]
    elif SIMULATION[7]=='R':
        model_name = 'RAMS'
    else:
        print('!!!!!issues with identifying model_name!!!!!')

    print('        model name: ',model_name)

    # grab the file needed
    if WHICH_TIME:
        if model_name=='RAMS':  
            selected_fil = variogram_helper_functions.find_RAMS_file(SIMULATION=SIMULATION,DOMAIN=DOMAIN,WHICH_TIME=WHICH_TIME)

        if model_name=='WRF':
            selected_fil =  variogram_helper_functions.find_WRF_file(SIMULATION=SIMULATION,DOMAIN=DOMAIN,WHICH_TIME=WHICH_TIME)
    
    if MODEL_FILE:
        print('a file path is provided')
        selected_fil = MODEL_FILE
    
    timestring = variogram_helper_functions.get_time_from_a_file(selected_fil,model_name)[0]
    print('time in this file = ',timestring)
    
    #### MAIN PART ####
    # read the main variable 
    z, z_name, z_units, z_time = read_vars_WRF_RAMS.read_variable(selected_fil,VARIABLE[0],model_name,output_height=False,interpolate=VARIABLE[1]>-1,level=VARIABLE[1],interptype=VARIABLE[2])
    
    # remove 40km from the edges
    z = variogram_helper_functions.remove_edges(z, 40.0, dx) # remove 40 km along each edge - new version
    
    # get the dimensions for producing list of random coordinates
    y_dim, x_dim     = np.shape(z)
    

    if CONDITION_INFO[0]=='environment':
        print('getting random coordinates over ',CONDITION_INFO[0],' points with threshold ',CONDITION_INFO[1])
        #print('        getting total condensate for conditional variogram') 
        if VARIABLE[1]<0:
            conditional_field_mask_file = '/home/isingh/code/variogram_data/'+VERSION+'/'+SIMULATION+'/G'+DOMAIN+'/masks/'+SIMULATION+'_'+CONDITION_INFO[0]+'_mask_QTC_threshold_'+str(CONDITION_INFO[1])+'_levtype_'+'model'+'_lev_0_'+z_time+'.npy'
        else: 
            conditional_field_mask_file = '/home/isingh/code/variogram_data/'+VERSION+'/'+SIMULATION+'/G'+DOMAIN+'/masks/'+SIMULATION+'_'+CONDITION_INFO[0]+'_mask_QTC_threshold_'+str(CONDITION_INFO[1])+'_levtype_'+VARIABLE[2]+'_lev_'+str(VARIABLE[1])+'_'+z_time+'.npy'

        print('reading mask file:',conditional_field_mask_file)
        conditional_field = np.load(conditional_field_mask_file)
        #conditional_field = variogram_helper_functions.remove_edges(conditional_field, 10.0, dx) - old version
        conditional_field = variogram_helper_functions.remove_edges(conditional_field, 40.0, dx) #- new version
        print('        min, max for the condensate field is ',np.min(conditional_field),' ',np.max(conditional_field))

        coords = variogram_helper_functions.produce_random_coords_conditional(SAMPLE_SIZE, conditional_field, CONDITION_STATEMENT=lambda x: x > -100)
        max_lag          = np.sqrt(x_dim**2 + y_dim**2)/2.0# in grid points
        
        
        #------- PLOT QTC FIELD ---------#
        # create total condensate plot
        
        if make_QTC_plot:
            savepng = '/home/isingh/code/variogram_data/'+VERSION+'/'+SIMULATION+'/'+'G'+DOMAIN+'/PNGs/'
            if not os.path.exists(savepng):
                os.makedirs(savepng)

            fig, ax = plt.subplots(1,figsize=(9,9))
            #qtc_field_contours = ax.imshow(conditional_field)
            qtc_field_contours = ax.imshow(z)

            y_coords, x_coords = zip(*coords)
            ax.scatter(np.array(x_coords), np.array(y_coords), color='red', marker='o',s=.07)
            plt.title('random pts (env) at level '+str(VARIABLE[1])+'\n'+z_time)
            plt.colorbar(qtc_field_contours)
            plt.savefig(savepng+'publication_40km_edges_'+VARIABLE[0]+'_levtype_'+VARIABLE[2]+'_lev_'+str(VARIABLE[1])+'_ensemble_'+str(ENSEMBLE_NUMBER)+'_QTC_field_environment_points_'+SIMULATION+'_'+z_time+'.png')
            plt.close()

    if CONDITION_INFO[0]=='storm': 
        print('getting random coordinates over ',CONDITION_INFO[0],' points with threshold ',CONDITION_INFO[1])
        print('        getting total condensate for conditional variogram')
        
        if VARIABLE[1]<0:
            conditional_field, _, _, _ = read_vars_WRF_RAMS.read_variable(selected_fil,'QTC',model_name,output_height=False,interpolate=True,level=0,interptype='model')
            #conditional_field          = variogram_helper_functions.remove_edges(conditional_field, 10.0, dx)
            conditional_field          = variogram_helper_functions.remove_edges(conditional_field, 40.0, dx)
        else:
            conditional_field, _, _, _ = read_vars_WRF_RAMS.read_variable(selected_fil,'QTC',model_name,output_height=False,interpolate=VARIABLE[1]>-1,level=VARIABLE[1],interptype=VARIABLE[2])
            #conditional_field          = variogram_helper_functions.remove_edges(conditional_field, 10.0, dx)
            conditional_field          = variogram_helper_functions.remove_edges(conditional_field, 40.0, dx)
        
        print('size of conditional field ',np.shape(conditional_field))
        # regionprops part
        # using region props to delineate storm objects
        img      = conditional_field
        mask     = img > CONDITION_INFO[1]
        mask     = morphology.remove_small_objects(mask, 32)
        #mask    = morphology.remove_small_holes(mask, 10)
        labels   = measure.label(mask)
        props    = measure.regionprops(labels, img)
        if len(props)==0:
            print('regionprops could not find any storm !!!\n\n\n')
            coords = []
        else:
            major_axis_length_list = []
            
            for index in range(1, labels.max()):
                major_axis_length_list.append(getattr(props[index], 'major_axis_length'))
                
                if make_variogram_for_each_storm:
                    print('estimating variogram for storm#',index)
                    max_lag_storm          = getattr(props[index], 'major_axis_length')/2.0
                    print('using maximum lag',max_lag_storm,' grid points')
                    coords_storm = props[index].coords
                    print('size of this storm = ',len(coords_storm))
                    #print('coordinates of this storm are: ',coords_storm)
                    #rows = [uu[0] for uu in chosen_area_label_coords_array]
                    #cols = [uu[1] for uu in chosen_area_label_coords_array]
                    #storm_mask_nans = np.zeros_like(img)*np.nan  # *np.nan
                    #storm_mask_nans[rows, cols] = 1.0
                    nonnan_coords_storm, nonnan_values_storm = variogram_helper_functions.get_values_at_random_coords(z, coords_storm)
                    num_lag_classses_storm = int(max_lag_storm*dx/BIN_WIDTH_IN_KM)
                    # create a variogram and save bin and variogram values in a matrix for saving
                    V_storm , bins_storm, exp_variogram_storm            = variogram_helper_functions.make_variogram(nonnan_coords_storm, nonnan_values_storm,num_lag_classses_storm,MAXLAG=max_lag_storm,DX=dx,ESTIMATOR=ESTIMATOR)
                    bins_middle_points_storm, counts_storm, widths_storm = variogram_helper_functions.retrieve_histogram(V_storm,DX=dx)
                    ##########################################################################################
                    # plot the coordinates used to make variogram for this storm object
                    savepng = '/home/isingh/code/variogram_data/'+SIMULATION+'/'+'G'+DOMAIN+'/PNGs/'
                    fig, ax = plt.subplots(1,figsize=(9,9))
                    z_field_contours = ax.imshow(z)
                    label_i = props[index].label
                    contour = measure.find_contours(labels == label_i, 0.5)[0]
                    y, x = contour.T
                    ax.plot(x, y, linewidth=1.0, color = 'k') # Plot the contour lines
                    y_coords_storm, x_coords_storm = zip(*coords_storm)
                    ax.scatter(np.array(x_coords_storm), np.array(y_coords_storm), color='red', marker='o',s=.07)
                    plt.title(VARIABLE[0]+ '('+VARIABLE[4]+') with random pts (storm#'+str(index)+')'+'\n'+z_time)
                    plt.colorbar(z_field_contours)
                    plt.savefig(savepng+'publication_'+VARIABLE[0]+'_storm_object'+str(index)+SIMULATION+'_'+z_time+'.png')
                    plt.close()
                    ##########################################################################################
                    if SAVE_CSV:
                        savecsv = '/home/isingh/code/variogram_data/'+VERSION+'/'+SIMULATION+'/'+'G'+DOMAIN+'/CSVs'
                        if not os.path.exists(savecsv):
                            os.makedirs(savecsv)
                        if VARIABLE[2]:
                            data_file_storm = savecsv+'/publication_individual_storm_'+str(index)+'_experimental_variogram_'+SAMPLING_STRATEGY+'_sampling_'+str(SAMPLE_SIZE)+'_samples_binwidth_'+str(int(BIN_WIDTH_IN_KM))+'km_estimator_'+ESTIMATOR+'_'+'_'+SIMULATION+'_G'+DOMAIN+'_'+CONDITION_INFO[0]+'_points_threshold_'+str(CONDITION_INFO[1])+'_'+VARIABLE[0]+'_levtype_'+VARIABLE[2]+'_lev_'+str(int(VARIABLE[1]))+'_'+z_time+'.csv'
                        else:
                            data_file_storm = savecsv+'/publication_individual_storm_'+str(index)+'_experimental_variogram_'+SAMPLING_STRATEGY+'_sampling_'+str(SAMPLE_SIZE)+'_samples_binwidth_'+str(int(BIN_WIDTH_IN_KM))+'km_estimator_'+ESTIMATOR+'_'+'_'+SIMULATION+'_G'+DOMAIN+'_'+CONDITION_INFO[0]+'_points_threshold_'+str(CONDITION_INFO[1])+'_'+VARIABLE[0]+'_levtype_'+'None'+'_lev_'+'None'+'_'+z_time+'.csv'

                        data_matrix_storm = np.column_stack((bins_storm, counts_storm, widths_storm, exp_variogram_storm))
                        np.savetxt(data_file_storm, data_matrix_storm, delimiter=',', header='bins,counts,widths,exp_variogram', comments='')
            ######################################################
          
            #print('list of all major lenth axes in gridpoints: ',major_axis_length_list)
            print('max length of a storm object in gridpoints = ',max(major_axis_length_list))
            print('max length in km = ',max(major_axis_length_list)*dx)

            max_lag          = max(major_axis_length_list)/2.0# in grid points
            #--------------------------------------------------
            print('        min, max for the condensate field is ',np.min(conditional_field),' ',np.max(conditional_field))
            coords = variogram_helper_functions.produce_random_coords_conditional(SAMPLE_SIZE, conditional_field, CONDITION_STATEMENT=lambda x: x >= CONDITION_INFO[1])
        #------- PLOT QTC FIELD ---------#
        # create total condensate plot
        if make_QTC_plot:
            savepng = '/home/isingh/code/variogram_data/'+VERSION+'/'+SIMULATION+'/'+'G'+DOMAIN+'/PNGs/'
            if not os.path.exists(savepng):
                os.makedirs(savepng)
                
            fig, ax = plt.subplots(1,figsize=(9,9))
            qtc_field_contours = ax.imshow(conditional_field)

            if len(props)>0:
                for index in range(1, labels.max()):
                        major_axis_length_list.append(getattr(props[index], 'major_axis_length'))
                        label_i = props[index].label
                        contour = measure.find_contours(labels == label_i, 0.5)[0]
                        # Plot the contour lines
                        y, x = contour.T
                        ax.plot(x, y, linewidth=1.0)

                y_coords, x_coords = zip(*coords)
                ax.scatter(np.array(x_coords), np.array(y_coords), color='red', marker='o',s=.07)

            plt.title('QTC field (kg/kg) with random pts (storm)'+'\n'+z_time)
            plt.colorbar(qtc_field_contours)
            plt.savefig(savepng+'publication_40km_edges_'+VARIABLE[0]+'_ensemble_'+str(ENSEMBLE_NUMBER)+'_QTC_field_storm_points_'+SIMULATION+'_'+z_time+'.png')
            plt.close()
        
    if CONDITION_INFO[0]=='all':
        print('getting random coordinates over ',CONDITION_INFO[0],' points')
        coords = variogram_helper_functions.produce_random_coords(x_dim,y_dim,SAMPLE_SIZE,SAMPLING_STRATEGY)   
        max_lag          = np.sqrt(x_dim**2 + y_dim**2)/2.0# in grid points
                
    ###########  VARIOGRAM ESTIMATION ##########   
    # get the values of the field at the random coordinates
    if len(coords)>0:
        nonnan_coords, nonnan_values = variogram_helper_functions.get_values_at_random_coords(z, coords)
        num_lag_classses = int(max_lag*dx/BIN_WIDTH_IN_KM)
        # create a variogram and save bin and variogram values in a matrix for saving
        V , bins, exp_variogram = variogram_helper_functions.make_variogram(nonnan_coords, nonnan_values,num_lag_classses,MAXLAG=max_lag,DX=dx,ESTIMATOR=ESTIMATOR)
        print('        using bin width = ',BIN_WIDTH_IN_KM,' km')
        bins_middle_points, counts, widths = variogram_helper_functions.retrieve_histogram(V,DX=dx)
    else:
        V                  = -999
        bins               = -999
        exp_variogram      = -999
        bins_middle_points = -999
        counts             = -999
        widths             = -999
        ##########################

    if SAVE_CSV:
        savecsv = '/home/isingh/code/variogram_data/'+VERSION+'/'+SIMULATION+'/'+'G'+DOMAIN+'/CSVs/'
        
        if not os.path.exists(savecsv):
            os.makedirs(savecsv)
            
        if VARIABLE[2]:
            data_file = savecsv+'publication_40km_edge_experimental_variogram_ensemble_'+str(ENSEMBLE_NUMBER)+'_'+SAMPLING_STRATEGY+'_sampling_'+str(SAMPLE_SIZE)+'_samples_binwidth_'+str(int(BIN_WIDTH_IN_KM))+'km_estimator_'+ESTIMATOR+'_'+SIMULATION+'_G'+DOMAIN+'_'+CONDITION_INFO[0]+'_points_threshold_'+str(CONDITION_INFO[1])+'_'+VARIABLE[0]+'_levtype_'+VARIABLE[2]+'_lev_'+str(int(VARIABLE[1]))+'_'+z_time+'.csv'
        else:
            data_file = savecsv+'publication_40km_edge_experimental_variogram_ensemble_'+str(ENSEMBLE_NUMBER)+'_'+SAMPLING_STRATEGY+'_sampling_'+str(SAMPLE_SIZE)+'_samples_binwidth_'+str(int(BIN_WIDTH_IN_KM))+'km_estimator_'+ESTIMATOR+'_'+SIMULATION+'_G'+DOMAIN+'_'+CONDITION_INFO[0]+'_points_threshold_'+str(CONDITION_INFO[1])+'_'+VARIABLE[0]+'_levtype_'+'None'+'_lev_'+'None'+'_'+z_time+'.csv'

        data_matrix = np.column_stack((bins, counts, widths, exp_variogram))
        np.savetxt(data_file, data_matrix, delimiter=',', header='bins,counts,widths,exp_variogram', comments='')

        print('        saving variogram data to ',data_file)
        #print('    ------\n')
    return #bins_middle_points, exp_variogram, coords
#------------------------------------------------------------------------------------------------------------------------
variables= [
            ['WSPD'  , 0   , 'model'   , '$wspd$'       , '$m^{2}s^{-2}$']   ,
            ['QV'    , 0   , 'model'   , '$q_{v}$'      , '$kg^{2}kg^{-2}$'] ,
            ['Tk'    , 0   , 'model'   , '$temperature$', '$K^{2}$']         ,
            ['WSPD'  , 750 , 'pressure', '$wspd$'       , '$m^{2}s^{-2}$']   ,
            ['QV'    , 750 , 'pressure', '$q_{v}$'      , '$kg^{2}kg^{-2}$'] ,
            ['Tk'    , 750 , 'pressure', '$temperature$', '$K^{2}$']         , 
            ['WSPD'  , 500 , 'pressure', '$wspd$'       , '$m^{2}s^{-2}$']   ,
            ['QV'    , 500 , 'pressure', '$q_{v}$'      , '$kg^{2}kg^{-2}$'] ,
            ['Tk'    , 500 , 'pressure', '$temperature$', '$K^{2}$']         
           ]

#variables= [['MCAPE'   , -999,  'model'     , '$MCAPE$'        , '$J^{2}kg^{-2}$'] ,]
#             ['SHF'   , -999,  'model'     , '$SHF$'        , '$W^{2}m^{-4}$'] ,
#             ['LHF'   , -999,  'model'     , '$LHF$'        , '$W^{2}m^{-4}$'] ,
#            ]      
     
domain='1'
colors      = ['#000000','#377eb8', '#56B4E9','#ff7f00', '#4daf4a','#f781bf', '#a65628', '#984ea3','#999999', '#e41a1c', '#dede00']
simulations = ['PHI2.1-R','PHI1.1-R','AUS1.1-R','USA1.1-R','WPO1.1-R','BRA1.1-R','BRA1.2-R','RSA1.1-R','ARG1.1-R','ARG1.2-R','DRC1.1-R']
thresholds  = [0.0000001,]
estimators  = ['matheron',]#'first_order_structure_function','second_order_structure_function','third_order_structure_function','fourth_order_structure_function','fifth_order_structure_function',]
#n',]#,
partitions  = ['environment',] #'storm','all'
sampling_strategy = 'randomly_distributed'
bin_widths  = [5.0,]
sample_sizes = [20000,]
ensembles = np.arange(1,2,1)
print('# ensembles:',ensembles)

file_dict = {'PHI2.1-R':'/monsoon/MODEL/LES_MODEL_DATA/V0/PHI2.1-R-V0/G3/out_30s/a-A-2019-09-10-153000-g'+domain+'.h5',
             'PHI1.1-R':'/monsoon/MODEL/LES_MODEL_DATA/V0/PHI1.1-R-V0/G3/out_30s/a-A-2019-09-10-120000-g'+domain+'.h5',
             'AUS1.1-R':'/monsoon/MODEL/LES_MODEL_DATA/V0/AUS1.1-R-V0/G3/out_30s/a-A-2006-01-23-120000-g'+domain+'.h5',
             'USA1.1-R':'/monsoon/MODEL/LES_MODEL_DATA/V0/USA1.1-R-V0/G3/out_30s/a-A-2022-09-16-110000-g'+domain+'.h5',
             'WPO1.1-R':'/monsoon/MODEL/LES_MODEL_DATA/V0/WPO1.1-R-V0/G3/out_30s/a-A-2018-08-28-065000-g'+domain+'.h5',
             'BRA1.1-R':'/monsoon/MODEL/LES_MODEL_DATA/V0/BRA1.1-R-V0/G3/out_30s/a-A-2014-03-31-190000-g'+domain+'.h5',
             'BRA1.2-R':'/monsoon/MODEL/LES_MODEL_DATA/V0/BRA1.2-R-V0/G3/out_30s/a-A-2014-04-01-140000-g'+domain+'.h5',
             'RSA1.1-R':'/monsoon/MODEL/LES_MODEL_DATA/V0/RSA1.1-R-V0/G3/out_30s/a-A-2008-03-11-123000-g'+domain+'.h5',
             'ARG1.1-R':'/monsoon/MODEL/LES_MODEL_DATA/V0/ARG1.1-R-V0/G3_old/out_30s/a-A-2018-12-13-210000-g'+domain+'.h5',
             'ARG1.2-R':'/monsoon/MODEL/LES_MODEL_DATA/V0/ARG1.2-R-V0/G3/out_30s/a-A-2018-12-14-013000-g'+domain+'.h5',
             'DRC1.1-R':'/monsoon/MODEL/LES_MODEL_DATA/V0/DRC1.1-R-V0/G3/out_30s/a-A-2016-12-30-113000-g'+domain+'.h5'}

# run with one core in serial
#save_variogram('V0',None,'/monsoon/MODEL/LES_MODEL_DATA/V0/DRC1.1-R-V0/G3/out_30s/a-A-2016-12-30-113000-g1.h5',variables[0],'DRC1.1-R', ['storm',0.0000001], 5000, 5.0, '1', True, 'matheron')

#Running on the terminal in parallel
argument = []

for simulation in simulations:
    for variable in variables:
        for partition in partitions:
            for threshold in thresholds:
                for estimator in estimators:
                    for bin_width in bin_widths:
                        for sample_size in sample_sizes:
                            for ensemble_member in ensembles:
                                argument = argument + [('V0', None, file_dict[simulation], variable, simulation,ensemble_member, [partition, threshold] , sample_size, bin_width, domain, True, estimator)]

print('length of argument is: ',len(argument))
for element in argument:
    print(element)
############################### FIRST OF ALL ################################
cpu_count1 = 16 #cpu_count()
print('number of cpus: ',cpu_count1)
# # #############################################################################

def main(FUNCTION, ARGUMENT):
    start_time = time.perf_counter()
    with Pool(processes = (cpu_count1)) as pool:
        data = pool.starmap(FUNCTION, ARGUMENT)
    finish_time = time.perf_counter()
    print(f"Program finished in {finish_time-start_time} seconds")
    
if __name__ == "__main__":
    main(save_variogram, argument)

# Fit variogram models

In [ ]:
import pandas as pd
from scipy.optimize import curve_fit
import numpy as np
import glob
import matplotlib.pyplot as plt
import matplotlib
from skgstat.models import spherical, exponential, gaussian, cubic
import matplotlib.ticker as ticker
import variogram_helper_functions

matplotlib.rcParams["font.family"] = "Roboto"
matplotlib.rcParams["font.sans-serif"] = ["Roboto"]  # For non-unicode text
matplotlib.rcParams['axes.labelsize'] = 25
matplotlib.rcParams['axes.titlesize'] = 20
matplotlib.rcParams['xtick.labelsize'] = 10
matplotlib.rcParams['ytick.labelsize'] = 25
matplotlib.rcParams['legend.fontsize'] = 25
matplotlib.rcParams['legend.facecolor'] = 'w'

simulations = ['ARG1.1-R', 'ARG1.2-R', 'PHI2.1-R', 'AUS1.1-R', 'DRC1.1-R', 'PHI1.1-R', 'USA1.1-R', 'WPO1.1-R', 'BRA1.1-R', 'BRA1.2-R', 'RSA1.1-R']

# List of variables with different names and levels
variables= [
             ['WSPD'  , 0   , 'model'   , '$wspd$'       , '$m^{2}s^{-2}$']   ,]
             ['QV'    , 0   , 'model'   , '$q_{v}$'      , '$kg^{2}kg^{-2}$'] ,
             ['Tk'    , 0   , 'model'   , '$temperature$', '$K^{2}$']         ,
             ['WSPD'  , 500 , 'pressure', '$wspd$'       , '$m^{2}s^{-2}$']   ,
             ['QV'    , 500 , 'pressure', '$q_{v}$'      , '$kg^{2}kg^{-2}$'] ,
             ['Tk'    , 500 , 'pressure', '$temperature$', '$K^{2}$']         ,
             ['MCAPE' , -999,  'model'  , '$MCAPE$'      , '$J^{2}kg^{-2}$']  ,
             ['SHF'   , -999,  'model'  , '$SHF$'        , '$W^{2}m^{-4}$']   ,
             ['LHF'   , -999,  'model'  , '$SHF$'        , '$W^{2}m^{-4}$']   ,
           ]



def linear(h, r, c0, b=0):
    return b + c0 * (h)

def power(h, r, c0, b=0):
    return b + c0 * (h**r)

def sph_exp(h,a1,b1,a2,b2):
    return spherical(h, a1, b1) + exponential(h, a2, b2)

def gau_sph(h,a1,b1,a2,b2):
    return gaussian(h, a1, b1)  + spherical(h, a2, b2)

def exp_gau(h,a1,b1,a2,b2):
    return exponential(h, a1, b1)  + gaussian(h, a2, b2)

curve_fitting_fs = {
    'gaussian': lambda h, a, b: gaussian(h, a, b),
    'exponential': lambda h, a, b: exponential(h, a, b),
    'spherical': lambda h, a, b: spherical(h, a, b),
    'cubic': lambda h, a, b: cubic(h, a, b),
    'power': lambda h, a, b: power(h, a, b),
    'sph_exp':lambda h, a1, b1, a2, b2: spherical(h, a1, b1) + exponential(h, a2, b2),
    'exp_gau':lambda h, a1, b1, a2, b2: exponential(h, a1, b1) + gaussian(h, a2, b2),
}

theoretical_fits = {
    'gaussian': gaussian,
    'exponential': exponential,
    'spherical': spherical,
    'cubic': cubic,
    'power': power,
    'sph_exp':sph_exp,
    'exp_gau':exp_gau,
}

domain = '1'
version='V0'

file_dict = {'PHI2.1-R':'/monsoon/MODEL/LES_MODEL_DATA/V0/PHI2.1-R-V0/G3/out_30s/a-A-2019-09-10-153000-g'+domain+'.h5',
             'PHI1.1-R':'/monsoon/MODEL/LES_MODEL_DATA/V0/PHI1.1-R-V0/G3/out_30s/a-A-2019-09-10-120000-g'+domain+'.h5',
             'AUS1.1-R':'/monsoon/MODEL/LES_MODEL_DATA/V0/AUS1.1-R-V0/G3/out_30s/a-A-2006-01-23-120000-g'+domain+'.h5',
             'USA1.1-R':'/monsoon/MODEL/LES_MODEL_DATA/V0/USA1.1-R-V0/G3/out_30s/a-A-2022-09-16-110000-g'+domain+'.h5',
             'WPO1.1-R':'/monsoon/MODEL/LES_MODEL_DATA/V0/WPO1.1-R-V0/G3/out_30s/a-A-2018-08-28-065000-g'+domain+'.h5',
             'BRA1.1-R':'/monsoon/MODEL/LES_MODEL_DATA/V0/BRA1.1-R-V0/G3/out_30s/a-A-2014-03-31-190000-g'+domain+'.h5',
             'BRA1.2-R':'/monsoon/MODEL/LES_MODEL_DATA/V0/BRA1.2-R-V0/G3/out_30s/a-A-2014-04-01-140000-g'+domain+'.h5',
             'RSA1.1-R':'/monsoon/MODEL/LES_MODEL_DATA/V0/RSA1.1-R-V0/G3/out_30s/a-A-2008-03-11-123000-g'+domain+'.h5',
             'ARG1.1-R':'/monsoon/MODEL/LES_MODEL_DATA/V0/ARG1.1-R-V0/G3_old/out_30s/a-A-2018-12-13-210000-g'+domain+'.h5',
             'ARG1.2-R':'/monsoon/MODEL/LES_MODEL_DATA/V0/ARG1.2-R-V0/G3/out_30s/a-A-2018-12-14-013000-g'+domain+'.h5',
             'DRC1.1-R':'/monsoon/MODEL/LES_MODEL_DATA/V0/DRC1.1-R-V0/G3/out_30s/a-A-2016-12-30-113000-g'+domain+'.h5'}

cmaps_dict = {'QV':['viridis','red'],'THETAV':['viridis','red'],'Tk':['viridis','red'],'WSPD':['viridis','red'],'W':['viridis','red'],'MCAPE':['viridis','red'],\
             'ITC':['viridis','red'],'SHF':['viridis','red'],'LHF':['viridis','red'],'SHF':['viridis','red'],'PCP_ACC':['viridis','red']}

condition_info = ['environment',1e-07]
#'linear': linear,
#'gau_sph':gau_sph,
# DataFrame to store results with columns for each variable's effective range and selected variogram
results_df = pd.DataFrame(index=simulations)

# Flag to control plotting
plot_figures = True  # Set to True to enable plotting

for var in variables:
    var_name = var[0]  # e.g., SHF, LHF, WSPD
    level = var[1]     # e.g., -999, 0, 500
    col_range_name = f"{var_name}_{level}_effective_range"  # Column for effective range
    col_model_name = f"{var_name}_{level}_selected_model"   # Column for selected variogram model
    results_df[col_range_name] = np.nan  # Initialize column for effective range
    results_df[col_model_name] = None  # Initialize column for selected model
    
    var_units = var[4]

    for sim in simulations:
        print('================ SIMULATION ================= ', sim)
        
        ###########################
        z_time = variogram_helper_functions.get_time_from_RAMS_file(file_dict[sim])[1]
        if var[1]<0:
            conditional_field_mask_file = '/home/isingh/code/variogram_data/'+version+'/'+sim+'/G'+domain+'/masks/'+sim+'_'+condition_info[0]+'_mask_QTC_threshold_'+str(condition_info[1])+'_levtype_'+'model'+'_lev_0_'+z_time+'.npy'
        else: 
            conditional_field_mask_file = '/home/isingh/code/variogram_data/'+version+'/'+sim+'/G'+domain+'/masks/'+sim+'_'+condition_info[0]+'_mask_QTC_threshold_'+str(condition_info[1])+'_levtype_'+var[2]+'_lev_'+str(var[1])+'_'+z_time+'.npy'
        
        env_mask = np.load(conditional_field_mask_file)
        
        cmap = cmaps_dict[var[0]][0]
        
        variogram_helper_functions.make_plan_view_RAMS_WRF(file_dict, var, sim, '1', cmap, 15000, False, env_mask, )
        ###########################
        
        if plot_figures:
            fig = plt.figure(figsize=(9, 9))
            ax = plt.gca()

        vario_files = sorted(glob.glob('/home/isingh/code/variogram_data/V0/' + sim + '/G1/CSVs/publication_experimental_variogram_ensemble_1_randomly_distributed_sampling_20000_samples_binwidth_5km_estimator_*matheron*_G1_environment_points_threshold_1e-07_' + var_name + '_levtype_' + var[2] + '_lev_' + str(level) + '_*.csv'))
        print('found file: ', vario_files)
        best_rmse = np.inf
        best_range = None
        best_model = None

        for csv_file in vario_files:
            df = pd.read_csv(csv_file)
            #print(df)

            x_orig = df.bins.values
            y_orig = df.exp_variogram.values
            x = x_orig#[x_orig<=100.]
            y = y_orig#[x_orig<=100.]
            #X = np.asarray([(_ + 1) for _ in x])
            sigma = None#np.exp(1. / X)
            #
            
            xi = x
            
            if var[0] == 'QV':
                y=y*1000.*1000.
                var_units = '$g^{2}kg^{-2}$'

            if plot_figures:
                plt.plot(x, y, '-o', label='experimental variogram')

            for fit_name, fit_function in theoretical_fits.items():
                print('theoretical fit : ', fit_name)
                if fit_name == 'power':
                    param_bounds = ([0, 0], [np.inf, 2])
                    cof, cov = curve_fit(curve_fitting_fs[fit_name], x, y, sigma=sigma,bounds=param_bounds)
                elif fit_name in ['sph_exp','gau_sph','exp_gau']:
                    param_bounds = ([0, 0, 0, 0], [np.max(x), np.max(y), np.max(x), np.max(y)])
                    cof, cov = curve_fit(curve_fitting_fs[fit_name], x, y, sigma=sigma,bounds=param_bounds)
                else:
                    cof, cov = curve_fit(curve_fitting_fs[fit_name], x, y,sigma=sigma, bounds=(0, (np.max(x), np.max(y))))
                
                yi = list(map(lambda x: fit_function(x, *cof), xi))
                rmse_c = np.sum((y - yi) ** 2)

                # Handle labels for combination models
                if fit_name in ['sph_exp', 'gau_sph','exp_gau']:
                    vario_range_a1 = np.round(cof[0], 1)  # First range (a1)
                    vario_range_a2 = np.round(cof[2], 1)  # Second range (a2)
                    vario_range = f"{vario_range_a1}, {vario_range_a2}"  # Combine ranges
                    
                    if vario_range_a1>0.90*max(x):
                        vario_range_a1 = np.nan
                    if vario_range_a2>0.90*max(x):
                        vario_range_a2 = np.nan
                    
                    # min([vario_range_a1,vario_range_a2])
                elif fit_name == 'linear' or fit_name == 'power':
                    vario_range = np.nan
                else:
                    vario_range = np.round(cof[0], 1)
                    if vario_range>0.90*max(x):
                        vario_range = np.nan

                if fit_name == best_model:
                    label_text = fit_name + ' - range=' + str(vario_range) + ' km; rmse=' + "{:.1e}".format(rmse_c) + ' units'
                    # Add the plot with bold label for the selected model
                    plt.plot(xi, yi, label=label_text, linewidth=2.5, color='black')
                else:
                    label_text = fit_name + ' - range=' + str(vario_range) + ' km; rmse=' + "{:.1e}".format(rmse_c) + ' units'
                    # Add the plot with regular label for other models
                    plt.plot(xi, yi, label=label_text)

                if rmse_c < best_rmse:
                    best_rmse = rmse_c
                    best_range = vario_range
                    best_model = fit_name

                print('coefficients are ', *cof)
                print('RMSE constrained for ' + fit_name + ' function:   %.2f' % rmse_c)

        # Store the best effective range and the best model for this variable and simulation
        results_df.loc[sim, col_range_name] = best_range
        results_df.loc[sim, col_model_name] = best_model
        
        
        if plot_figures:
            # In the legend section, specify the font properties for bold text:
            handles, labels = ax.get_legend_handles_labels()
            legend_labels = []
            for label in labels:
                if best_model in label:
                    legend_labels.append(label)  # Use this to identify the selected model
                else:
                    legend_labels.append(label)

            # Set bold for the selected model in the legend
            legend = plt.legend(handles=handles, labels=legend_labels, loc='lower right', fontsize=12)
            for text in legend.get_texts():
                if best_model in text.get_text():
                    text.set_fontweight('bold')  # Bold the selected model's label
            plt.xlabel('lag distance (km)')
            plt.ylabel('Variogram')
            
            if var[1]>=0:
                plt.title('Variogram for ' + sim + '\n' + var[3] + ' at ' + var[2] + ' level ' + str(level) + ' (' + var[4] + ')')
            else:
                plt.title('Variogram for ' + sim + '\n' + var[3] + ' (' + var[4] + ')')
                
            intervals = 60
            # Changing tick frequencies
            ax.xaxis.set_major_locator(ticker.MultipleLocator(intervals))
            plt.xlim([0,max(x)])
            plt.grid()
            
            png_filename = 'theoretical_model_fit_'+sim+'_'+var[0]+'_'+var[2]+'_level_' + str(level)+'.png'
            print('saving to png ',png_filename)
            #plt.savefig(png_filename,dpi=150)
            #plt.close()
            
        print('------------------------\n\n')

# Output the final DataFrame
print(results_df)

# Compute structure function exponents

In [ ]:
from scipy.optimize import curve_fit
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import matplotlib

matplotlib.rcParams["font.family"]      = "Roboto"
matplotlib.rcParams["font.sans-serif"]  = ["Roboto"]  # For non-unicode text
matplotlib.rcParams['axes.labelsize']   = 25
matplotlib.rcParams['axes.titlesize']   = 20
matplotlib.rcParams['xtick.labelsize']  = 25
matplotlib.rcParams['ytick.labelsize']  = 25
matplotlib.rcParams['legend.fontsize']  = 25
matplotlib.rcParams['legend.facecolor'] = 'w'


colors_dict = {'PHI2.1-R':['#999999','-'],
               'PHI1.1-R':['#999999','--'],
               'AUS1.1-R':['#56B4E9','-'],
               'USA1.1-R':['#ff7f00','-'],
               'WPO1.1-R':['#4daf4a','-'],
               'BRA1.1-R':['#000000','-'],
               'BRA1.2-R':['#000000','--'],
               'RSA1.1-R':['#984ea3','-'],
               'ARG1.1-R':['#a65628','-'],
               'ARG1.2-R':['#a65628','--'],
               'DRC1.1-R':['#dede00','-'] } #,

simulations = ['PHI2.1-R','AUS1.1-R','DRC1.1-R','PHI1.1-R','USA1.1-R','WPO1.1-R','BRA1.1-R','BRA1.2-R','RSA1.1-R','ARG1.1-R','ARG1.2-R']
vertical_offset = 0.1
orders = ['first','second','third','fourth','fifth',]
df_slope = pd.DataFrame(index=simulations, columns=orders)
df_error = pd.DataFrame(index=simulations, columns=orders)

varrs =    [['WSPD'  , 500 , 'pressure', '$wspd$' ,'$m^{2}s^{-2}$']  ,]#['WSPD'  , 0   , 'model'   , '$wspd$'       , '$m^{2}s^{-2}$'] ,]
for var in varrs:
    for order in orders:
        fig = plt.figure(figsize=(9,9))
        ax = plt.gca()
        
        handles = []
        
        for ii, sim in enumerate(simulations):
             
            df_list = []

            vario_files = sorted(glob.glob('/home/isingh/code/variogram_data/V0/'+sim+'/G1/CSVs/publication_experimental_variogram_ensemble_*_randomly_distributed_sampling_20000_samples_binwidth_5km_estimator_'+order+'_order*_G1_environment_points_threshold_1e-07_'+var[0]+'_levtype_'+var[2]+'_lev_'+str(var[1])+'_*.csv'))
            print(vario_files)
            for csv_file in vario_files:
                df = pd.read_csv(csv_file)

            # Define a linear model for curve fitting
            def linear_model(x, a, b):
                return a * x + b
            
            df_filtered = df[(df['bins'] > 9) & (df['bins'] < 101)]
            # Use log10 of bins and exp_variogram for curve fitting
            log_bins = np.log10(df_filtered['bins'])
            log_exp_variogram = np.log10(df_filtered['exp_variogram'])

            # Fit the data
            params, cov = curve_fit(linear_model, log_bins, log_exp_variogram)
            slope, intercept = params
            
            slope_var = cov[0, 0]  # Variance of the slope
            

            # Calculate the fitted line
            fitted_line = linear_model(log_bins, slope, intercept)
          
            # calculate R^2 
            residuals = log_exp_variogram - linear_model(log_bins,  slope, intercept)
            ss_res    = np.sum(residuals**2)
            #You can get the total sum of squares (ss_tot) with
            ss_tot    = np.sum((log_exp_variogram-np.mean(log_exp_variogram))**2)
            #And finally, the r_squared-value with,
            r_squared = 1 - (ss_res / ss_tot)
            
            df_slope.loc[sim, order] = slope
            df_slope.loc[sim, 'r_squared_'+order] = r_squared
            df_error.loc[sim, order] = np.sqrt(slope_var)
            
    df_slope.to_csv('./structure_function_distance_log_slopes_'+var[0]+'_'+var[2]+'_level_'+str(var[1])+'.csv')  
    print(df_slope)